## [인터랙션] LightGCN 에서 뽑아온 임베딩 벡터로 annoy

In [1]:
import pandas as pd

In [2]:
data_path = "/opt/ml/.jupyter/lab/workspaces/data/"
ratings = pd.read_csv(data_path+"ml-25m/ratings.csv")
movies = pd.read_csv(data_path + "ml-25m/movies.csv")

ratings = ratings.merge(movies[["movieId","title"]],how="left",on = "movieId")
ratings['rating'] = 1

In [3]:
ratings.head()

,userId,movieId,rating,timestamp,title
0,1,296,1,1147880044,Pulp Fiction (1994)
1,1,306,1,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994)
2,1,307,1,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993)
3,1,665,1,1147878820,Underground (1995)
4,1,899,1,1147868510,Singin' in the Rain (1952)


In [4]:
mbti = pd.read_csv("/opt/ml/.jupyter/lab/workspaces/data/inter_mbti_ml_title_movieId.csv")

In [5]:
mbti

,title,movieId
0,moana,73141
1,inside out,122470
2,zootopia,152081
3,legally blonde,4447
4,dead poets society,1246
...,...,...
3224,checkered ninja,203298
3225,turma da mônica: laços,203745
3226,"where'd you go, bernadette",204702
3227,downton abbey,205076


In [6]:
user2idx = {k:v for v,k in enumerate(ratings['userId'].unique())}
item2idx = {k:v for v,k in enumerate(ratings['movieId'].unique())}
idx2item = {v:k for v,k in enumerate(ratings['movieId'].unique())}

In [7]:
ratings['useridx'] = ratings['userId'].map(user2idx) 
ratings['movieidx'] = ratings['movieId'].map(item2idx) 

In [8]:
ratings.head()

,userId,movieId,rating,timestamp,title,useridx,movieidx
0,1,296,1,1147880044,Pulp Fiction (1994),0,0
1,1,306,1,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),0,1
2,1,307,1,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),0,2
3,1,665,1,1147878820,Underground (1995),0,3
4,1,899,1,1147868510,Singin' in the Rain (1952),0,4


ratings의 유저, 영화 임베딩을 뽑기 위해 train_data.inter 파일을 만드는 과정

In [17]:
# 임베딩을 뽑기 위해 train_data.inter 파일 만들기 쉘
# ratings_for_lgcn = ratings[['userId', 'movieId', 'timestamp']]
# ratings_for_lgcn.columns = ['userId:token', 'movieId:token', 'timestamp:float']
# ratings_for_lgcn

,userId:token,movieId:token,timestamp:float
0,1,296,1147880044
1,1,306,1147868817
2,1,307,1147868828
3,1,665,1147878820
4,1,899,1147868510
...,...,...,...
25000090,162541,50872,1240953372
25000091,162541,55768,1240951998
25000092,162541,56176,1240950697
25000093,162541,58559,1240953434


In [18]:
# 임베딩을 뽑기 위해 train_data.inter 파일 만들기 쉘
# ratings_for_lgcn.to_csv("train_data.inter",sep='\t',index=False)

위의 인터랙션으로 학습된 모델에서 임베딩 가져오기

In [9]:
emb = pd.read_csv("/opt/ml/.jupyter/lab/workspaces/BaseLine/emb_list.csv", header=None)

In [10]:
emb

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.002425,-0.005020,-0.002396,-0.002710,-0.005918,-0.000091,0.001614,-0.000355,-0.001832,0.003243,...,0.002055,-0.000659,-0.005944,0.003073,-0.000935,-0.004552,-0.002956,0.006067,-0.004683,-0.002283
1,-0.419613,0.329477,-0.416849,-0.367531,0.356896,0.469562,-0.356054,0.473384,0.409519,0.474743,...,0.362406,-0.359313,-0.350114,0.348255,-0.410161,-0.385093,0.352926,-0.422700,-0.368102,0.406651
2,-0.268988,0.287859,-0.284004,-0.291609,0.270063,0.167634,-0.285118,0.175499,0.241801,0.170318,...,0.268751,-0.257847,-0.261998,0.280394,-0.215873,-0.328595,0.303511,-0.210801,-0.242678,0.234939
3,-0.031271,0.227388,0.077158,-0.021954,0.290896,0.663348,-0.178687,0.584792,0.254705,0.589111,...,0.239965,-0.258286,-0.332271,0.179603,-0.442636,0.311491,0.257483,-0.571436,-0.419248,0.278454
4,0.051184,0.181506,0.129626,0.004287,0.229944,0.636447,-0.138833,0.574531,0.190216,0.576395,...,0.187755,-0.190030,-0.278590,0.119494,-0.421519,0.303298,0.194130,-0.526388,-0.383999,0.225446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221585,0.366406,-0.303594,0.362098,0.323604,-0.323457,-0.289040,0.328368,-0.316090,-0.334187,-0.315628,...,-0.319740,0.316728,0.318225,-0.321170,0.324161,0.358830,-0.317624,0.305512,0.306109,-0.339409
221586,0.350672,-0.246470,0.358244,0.329455,-0.318132,-0.337550,0.308039,-0.346451,-0.349847,-0.350017,...,-0.315853,0.306287,0.297611,-0.297679,0.355928,0.348369,-0.282764,0.367955,0.337699,-0.349628
221587,0.454765,-0.348607,0.452943,0.400124,-0.381536,-0.422293,0.383423,-0.428962,-0.455152,-0.445997,...,-0.395328,0.389951,0.374390,-0.390836,0.438134,0.439666,-0.380095,0.429791,0.385250,-0.437122
221588,0.362287,-0.327534,0.371949,0.343913,-0.350839,-0.327840,0.349587,-0.332429,-0.364429,-0.351961,...,-0.348212,0.354808,0.330077,-0.353073,0.346380,0.388555,-0.331034,0.341219,0.345211,-0.358092


In [11]:
emb_list = emb.values.tolist()

In [12]:
len(emb_list)

221590

불러온 임베딩 벡터를 각각 아이템 임베딩 벡터, 유저 임베딩 벡터로 나눠주기

In [13]:
movieId_len = ratings['movieId'].nunique()

In [14]:
item_vecs = emb_list[1:movieId_len+1]
user_vecs = emb_list[movieId_len+2:]

In [15]:
len(item_vecs)

59047

In [16]:
len(user_vecs)

162541

annoy

In [30]:
import annoy

In [37]:
embedding_size = 64
similar_items_index = annoy.AnnoyIndex(embedding_size,'angular')

In [32]:
mbti['movieidx'] = mbti['movieId'].map(item2idx)

In [33]:
mbti[mbti['movieidx'].isnull()]

,title,movieId,movieidx
220,marry me,150834,NaN
270,the three musketeers,150106,NaN
432,the intruder,151939,NaN
435,the kitchen,138086,NaN
671,the last of the mohicans,167876,NaN
1370,the coldest game,208655,NaN
1434,lost,132512,NaN
1482,the father,187779,NaN
1877,the bounty hunter,125179,NaN
2030,homeward bound,161694,NaN


In [34]:
mbti.dropna(inplace = True)
mbti['movieidx'] = mbti['movieidx'].astype(int)

In [35]:
mbti.head()

,title,movieId,movieidx
0,moana,73141,20792
1,inside out,122470,24215
2,zootopia,152081,905
3,legally blonde,4447,358
4,dead poets society,1246,107


In [40]:
n_trees = 40
for idx in mbti['movieidx']:
    similar_items_index.add_item(idx2item[idx],item_vecs[idx]) 
similar_items_index.build(n_trees)

True

In [41]:
similar_items_index.save("similarity")

True

In [42]:
def similar_item(id,N):
    result = []
    neighbour,dist = similar_items_index.get_nns_by_item(id, N, include_distances=True)
    for i in range(0,N):
        result.append(mbti[mbti['movieId']==neighbour[i]]['title'].values[0])
    return result[1:N]

In [43]:
movieId_list = mbti['movieId'].unique()
result = similar_item(movieId_list[1],10)

In [47]:
mbti[mbti['movieId'] == movieId_list[1]]['title']

1    inside out
Name: title, dtype: object

In [44]:
result

['uhf',
 'flubber',
 'the autopsy of jane doe',
 'the monster',
 'faculty, the',
 'limitless',
 'boy',
 'on body and soul',
 'alice in wonderland']